In [1]:
import geopandas as gpd
import xarray as xr
import pandas as pd
import numpy as np 
from shapely.geometry import Point, Polygon, mapping

In [2]:
# Load .shp file with impact regions 
ir = gpd.read_file("C:\\Users\\Nayeli\\Documents\data\\2_projection\\1_regions\\ir_shp\\impact-region.shp")

# Load temperature data file
model = 'ACCESS-CM2'
file_path = 'C:\\Users\\Nayeli\\Downloads\\New Climate Models\\tas_day_ACCESS-CM2_ssp585_r1i1p1f1_gn_20150101-20640705_v20191108.nc' 
tas_SSP = xr.open_dataset(f'{file_path}')
temperatura = tas_SSP['tas']

In [4]:
# Calculate manually lat and lon dimensions
latitud = ((temperatura.lat.values[1]-temperatura.lat.values[0]) + (temperatura.lat.values[2]-temperatura.lat.values[1]))/2
longitud = ((temperatura.lon.values[1]-temperatura.lon.values[0]) + (temperatura.lon.values[2]-temperatura.lon.values[1]))/2
print(latitud, longitud)

1.25 1.875


In [5]:
def create_square(lon, lat):
    return Polygon([
        (lon, lat),
        (lon + longitud, lat),
        (lon + longitud, lat + latitud),
        (lon, lat + latitud)
    ])

# Obtaining the lon and lat values
new_lon = temperatura.lon.values
new_lat = temperatura.lat.values
# Converting lon to right -180 to 180 degrees
new_lon = np.where(new_lon > 180, new_lon - 360, new_lon)
# Create meshgrid
lon2d, lat2d = np.meshgrid(new_lon, new_lat)
# Flatten the values
lon_flatten = lon2d.flatten()
lat_flatten = lat2d.flatten()
# Create dataframe with lat and lon
points_df = pd.DataFrame({ 'longitude': lon_flatten, 'latitude': lat_flatten})
# Make the geometry with squares for the temperature values
points_df['geometry'] = [create_square(lon, lat) for lon, lat in zip(points_df.longitude, points_df.latitude)]
# Convert the DataFrame to GeoDataFrame
points_gdf = gpd.GeoDataFrame(points_df)
points_gdf = points_gdf.reset_index()
# Set same CRS
points_gdf.crs = ir.crs  

#Make the spatial join only once and recover the indices
relationship = gpd.sjoin(points_gdf, ir, how='inner', predicate='intersects')
relationship = relationship[['geometry','index_right']]

In [6]:
# Select a day to merge the temperature data into the impact region level
def new_response_function(day):
  
    temperatures = temperatura.sel(time=day).values.flatten() - 273.15
    # Associat
    relationship['temperature'] = temperatures[relationship.index]
    result = relationship.groupby('index_right')['temperature'].mean()
    print(day)
    
    return result

In [7]:
def create_file(model,year, scenario):
    fechas = temperatura['time'].values
    date_list = fechas[np.isin(fechas.astype('datetime64[Y]'), np.datetime64(f'{year}', 'Y'))]
    date_list = date_list.astype('datetime64[D]').astype(str)

    df = pd.DataFrame(ir['hierid'])
    for item in date_list:
        df[item] = new_response_function(item)
    df_rounded = df.round(1)
    df_rounded.to_csv(f'C:\\Users\\Nayeli\\Documents\\{model}\\SSP{scenario}\\{model}_SSP{scenario}_{year}.csv')
    #df_rounded.to_csv(f'D:\\Climate models\\{model}\\SSP{scenario}\\{model}_SSP{scenario}_{year}.csv')

In [8]:
scenario = 585
start_year = 2015
#end_year = start_year
years = np.arange(start_year, 2065)
for year in years:
    tas_SSP = xr.open_dataset(f'C:\\Users\\Nayeli\\Downloads\\New Climate Models\\tas_day_ACCESS-CM2_ssp585_r1i1p1f1_gn_20150101-20640705_v20191108.nc')
    temperatura = tas_SSP['tas']
    create_file(model, year, scenario)

2015-01-01
2015-01-02
2015-01-03
2015-01-04
2015-01-05
2015-01-06
2015-01-07
2015-01-08
2015-01-09
2015-01-10
2015-01-11
2015-01-12
2015-01-13
2015-01-14
2015-01-15
2015-01-16
2015-01-17
2015-01-18
2015-01-19
2015-01-20
2015-01-21
2015-01-22
2015-01-23
2015-01-24
2015-01-25
2015-01-26
2015-01-27
2015-01-28
2015-01-29
2015-01-30
2015-01-31
2015-02-01
2015-02-02
2015-02-03
2015-02-04
2015-02-05
2015-02-06
2015-02-07
2015-02-08
2015-02-09
2015-02-10
2015-02-11
2015-02-12
2015-02-13
2015-02-14
2015-02-15
2015-02-16
2015-02-17
2015-02-18
2015-02-19
2015-02-20
2015-02-21
2015-02-22
2015-02-23
2015-02-24
2015-02-25
2015-02-26
2015-02-27
2015-02-28
2015-03-01
2015-03-02
2015-03-03
2015-03-04
2015-03-05
2015-03-06
2015-03-07
2015-03-08
2015-03-09
2015-03-10
2015-03-11
2015-03-12
2015-03-13
2015-03-14
2015-03-15
2015-03-16
2015-03-17
2015-03-18
2015-03-19
2015-03-20
2015-03-21
2015-03-22
2015-03-23
2015-03-24
2015-03-25
2015-03-26
2015-03-27
2015-03-28
2015-03-29
2015-03-30
2015-03-31
2015-04-01

C:\Users\Nayeli\AppData\Local\Temp\ipykernel_10364\759290320.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[item] = new_response_function(item)


2015-04-11


C:\Users\Nayeli\AppData\Local\Temp\ipykernel_10364\759290320.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[item] = new_response_function(item)


2015-04-12


...

2017-05-04


C:\Users\Nayeli\AppData\Local\Temp\ipykernel_10364\759290320.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[item] = new_response_function(item)


2017-05-05


C:\Users\Nayeli\AppData\Local\Temp\ipykernel_10364\759290320.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[item] = new_response_function(item)


KeyboardInterrupt: 